In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn import decomposition
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import time

In [35]:
pip install git+https://github.com/samueljackson92/coranking.git

  Cloning https://github.com/samueljackson92/coranking.git to /tmp/pip-req-build-se0mm0aj
  Running command git clone -q https://github.com/samueljackson92/coranking.git /tmp/pip-req-build-se0mm0aj
  Created wheel for coranking: filename=coranking-0.1.1-cp36-cp36m-linux_x86_64.whl size=89371 sha256=126247cf0033e581ec42fe35c3681ed16e2248ca344506e04040ef3b6abd41d1
  Stored in directory: /tmp/pip-ephem-wheel-cache-52xvx2yx/wheels/8e/e0/11/4ae911b6fc472b8bf6087e29d861d2b798c119603adf4b93fa
Successfully built coranking


In [0]:
import coranking
from coranking.metrics import trustworthiness, continuity, LCMC


###1. Loading and Processing MNIST

In [3]:
link1='https://drive.google.com/open?id=1LAXOvFLyMmK8Pkdw6efWhGHtIQpkGnYd'

fluff, id = link1.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv') 
training_data  = pd.read_csv('train.csv')

1LAXOvFLyMmK8Pkdw6efWhGHtIQpkGnYd


In [0]:
labels  = training_data['label']
training_data = training_data.drop("label",axis=1)

In [5]:
print(training_data.shape)

(42000, 784)


In [6]:
from sklearn.preprocessing import StandardScaler
standardized_data = StandardScaler().fit_transform(training_data)
print(standardized_data.shape)

(42000, 784)


In [7]:
covar_matrix = np.matmul(standardized_data.T , standardized_data)

print ( "The shape of variance matrix = ", covar_matrix.shape)

The shape of variance matrix =  (784, 784)


In [8]:
training_data

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


###Decision Tree on Original Data

In [9]:
from sklearn import tree

X_train, X_test, y_train, y_test = train_test_split(training_data, labels, test_size=0.25, random_state=49)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.8484761904761905

###MDS

In [0]:
from sklearn.manifold import MDS
embedding = MDS(n_components=16)
X_transformed = embedding.fit_transform(training_data[:5000])

In [14]:
X_transformed.shape

(5000, 16)

###Decision T on MDS Data

In [0]:
from prettytable import PrettyTable
t_mds_10k = PrettyTable(['DT_score_onMDSData', 'Dimensions', 'MDS_time','DecisionTree_time' ,'total_time', 'trust_mds','cont_mds' ,'lcmc_mds'])


def MDS_Decision_Tree (training_data, components,size):
  #Reducing to mentioned components
  mds_start_time = time.time()
  embedding = MDS(n_components=components)
  training_data_mds = embedding.fit_transform(training_data[:size])
  mds_time = time.time() - mds_start_time
  
  
  #Applying Decision Tree
  X_train, X_test, y_train, y_test = train_test_split(training_data_mds, labels[:size], test_size=0.25, random_state=49)
  clf_start_time = time.time()
  clf = tree.DecisionTreeClassifier()
  clf = clf.fit(X_train, y_train)
  clf_time = time.time() - clf_start_time
  score=clf.score(X_test,y_test)
  score = str(round(score, 4))

  total_time = mds_time+clf_time
  mds_time = str(round(mds_time, 4))
  clf_time = str(round(clf_time, 4))
  total_time = str(round(total_time, 4))

  Q = coranking.coranking_matrix(training_data[:size], training_data_mds)
  trust_mds = trustworthiness(Q, min_k=1, max_k=50).sum()/50
  cont_mds = continuity(Q, min_k=1, max_k=50).sum()/50
  lcmc_mds = LCMC(Q, min_k=1, max_k=50).sum()/50

  trust_mds = str(round(trust_mds, 4))
  cont_mds = str(round(cont_mds, 4))
  lcmc_mds = str(round(lcmc_mds, 4))

  t_mds_10k.add_row([score, components, mds_time, clf_time ,total_time, trust_mds,cont_mds ,lcmc_mds])
  return

In [0]:
dimensions=784
size=9999
while dimensions >= 2:
  MDS_Decision_Tree (training_data, dimensions,size)
  dimensions=int(dimensions/2)

print(t_mds_10k)

In [1]:
print(t_mds_10k)

NameError: ignored

In [0]:
  n_components=10
  size = 5000
  mds_start_time = time.time()
  embedding = MDS(n_components=n_components)
  X_transformed = embedding.fit_transform(training_data[:size])
  mds_time = time.time() - mds_start_time
  


  
  
  
  #Applying Decision Tree

  X_train, X_test, y_train, y_test = train_test_split(training_data[:size], labels[:size], test_size=0.25, random_state=49)

  clf_start_time = time.time()
  clf = tree.DecisionTreeClassifier()
  clf = clf.fit(X_train, y_train)
  clf_time = time.time() - clf_start_time
  score=clf.score(X_test,y_test)
  score = str(round(score, 4))

  total_time = mds_time+clf_time
  mds_time = str(round(mds_time, 4))
  clf_time = str(round(clf_time, 4))
  total_time = str(round(total_time, 4))

  t1 = PrettyTable(['DT_score', 'Dimensions', 'MDS_time','DecisionTree_time', 'total_time',])
  
  t1.add_row([score, n_components , mds_time, clf_time ,total_time ])

In [29]:
print(t1)

+----------+------------+----------+-------------------+------------+
| DT_score | Dimensions | MDS_time | DecisionTree_time | total_time |
+----------+------------+----------+-------------------+------------+
|  0.7376  |     10     | 835.8946 |       0.584       |  836.4786  |
+----------+------------+----------+-------------------+------------+


In [27]:
print(t1)

+----------+------------+----------+-------------------+------------+
| DT_score | Dimensions | MDS_time | DecisionTree_time | total_time |
+----------+------------+----------+-------------------+------------+
|  0.604   |     10     | 29.9837  |       0.1163      |    30.1    |
+----------+------------+----------+-------------------+------------+


In [25]:
print(t1)

+----------+------------+----------+-------------------+------------+
| DT_score | Dimensions | MDS_time | DecisionTree_time | total_time |
+----------+------------+----------+-------------------+------------+
|  0.572   |    150     | 45.9175  |       0.1184      |  46.0359   |
+----------+------------+----------+-------------------+------------+
